# Patent Kaggle challenge

notebook from fastai course lesson 4

## Importing kaggle dataset

In [3]:
!kaggle competitions download -c us-patent-phrase-to-phrase-matching
!unzip us-patent-phrase-to-phrase-matching.zip -d us-patent-phrase-to-phrase-matching
!rm us-patent-phrase-to-phrase-matching.zip

In [6]:
from pathlib import Path

path = Path('us-patent-phrase-to-phrase-matching')
!ls {path}

sample_submission.csv  test.csv  train.csv


## Viewing data

In [10]:
import pandas as pd
df = pd.read_csv(path/'train.csv')
df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [11]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


In [13]:
df['input'] = 'TEXT1: ' + df.anchor + '; TEXT2: ' + df.target + '; CONTEXT: ' + df.context
df.input.head()

0    TEXT1: abatement; TEXT2: abatement of pollutio...
1    TEXT1: abatement; TEXT2: act of abating; CONTE...
2    TEXT1: abatement; TEXT2: active catalyst; CONT...
3    TEXT1: abatement; TEXT2: eliminating process; ...
4    TEXT1: abatement; TEXT2: forest region; CONTEX...
Name: input, dtype: object

## Tokenisation

In [17]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [22]:
model_nm = 'microsoft/deberta-v3-small'

In [26]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

ValueError: Converting from Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast convertors: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'RealmTokenizer', 'Refor